Select the Python 3 kernel if you are prompted for a kernel.

### Multi-Agent Systems with Strands Agents leveraging Strands custom tools, built-in tools and MCP tools


The **Strands Agents SDK** supports different multi-agent patterns such as:

- [**Swarm**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/swarm/)  
- [**Graph**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/graph/)  
- [**Workflow**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/workflow/)  
- [**Agent as a Tool**](https://strandsagents.com/latest/user-guide/concepts/multi-agent/agents-as-tools/)

**Note**
In this notebook we will be using **Graph** pattern but we will not be using the **agent_graph** built-in Strands tool. Strands Agents SDK provides a built-in agent_graph tool that simplifies multi-agent system implementation. We will look at the **agent_graph** implementation in the **[02_strands_graph_coordinator_agents.ipynb](./02_strands_graph_coordinator_agents.ipynb)** notebook next. 

### Tool Architecture Strategy

#### Local Tool Integration
- Custom tools tightly coupled to a specific agent's functionality
- Simpler deployment with no external dependencies
- Direct control over tool behavior and updates
- Lower latency due to local execution

#### Centralized Tool Server
- Tools shared across multiple agents or frameworks
- Single source of truth for tool functionality
- Efficient scaling and maintenance

#### Key Considerations:
- **Development efficiency:** Centralized tools reduce duplicate code
- **Maintenance:** Single update point for shared tools
- **Scalability:** Better resource utilization across agents
- **Consistency:** Standardized tool behavior across implementations


In this notebook, we will be providing the credit check functionality exposed as a tool through an MCP server and integrate with Strands custom and built-in tools.

## 1. Setup and Installation

First, let's install the required libraries:

In [ ]:
%pip install --upgrade -q -r ../src/requirements.txt

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## 2. Import Libraries and Configure Logging

In [ ]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent
import sys
import logging
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client



from strands import Agent, tool
from strands_tools import retrieve, calculator



In [ ]:
# Set up logging specifically for Strands components
loggers = [
  'strands',
  'strands.agent',
  'strands.tools',
  'strands.models',
  'strands.bedrock'
]
for logger_name in loggers:
  logger = logging.getLogger(logger_name)
  logger.setLevel(logging.INFO)
  # Add console handler if not already present
  if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

## 3. Configure Knowledge Base

First you need to create the Knowledge base before creating an agent to answer questions on mortages. To create the Amazon Bedrock Knowledge Base, you need to run this notebook first if not done already [01_create_knowledgebase.ipynb](../2_bedrock-multi-agent/01_create_knowledgebase.ipynb). When you run the below cell you should be able to see the id of the Knowledge base that you created.

In [ ]:
%store -r kb_id
print("KnowledgeBase ID:",kb_id)

The Strands *retrieve** tool can access the Knowledge Base you have created via the **KNOWLEDGE_BASE_ID** environment variable

In [ ]:
# Set knowledge base ID as environment variable so that Strands retrieve tool can use it
os.environ["KNOWLEDGE_BASE_ID"] = kb_id
os.environ["AWS_REGION"] = 'us-east-1'


## 4. Create Agent for General Mortgage Questions

We will be creating an agent to answer general mortage questions providing it the **retrieve** tool to access the Knowledge Base created earlier.

In [ ]:
@tool
def answer_general_mortgage_questions(query):
    # Create the General Mortgage Agent
    general_mortgage_agent = Agent(
        model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        tools=[
           retrieve
        ],
        system_prompt="""
        You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types. Greet the customer first.
        
        IMPORTANT: Always use the retrieve tool to search the knowledge base before answering any mortgage-related questions.
        
        You can:
        1. Provide general information about mortgages
        2. Handle conversations about general mortgage questions, like high level concepts of refinancing or tradeoffs of 15-year vs 30-year terms.
        3. Offer guidance on the mortgage refinancing and tradeoffs of mortgage types.
        4. Access a knowledge base of mortgage information using the retrieve tool
        5. Only answer from the knowledge base and not from your general knowledge. If you dont have the answer from Knowledge base, say "I dont know"
        
        When helping users:
        - ALWAYS call the retrieve tool first to search for relevant information
        - Provide clear explanations based on retrieved information
        - Use plain language to explain complex financial terms
        - Offer balanced advice considering both pros and cons
        - Be informative without making specific financial recommendations
        
        Remember that you're providing general mortgage information, not financial advice.
        Always clarify that users should consult with a financial advisor for personalized advice.
        """
    )
    return str(general_mortgage_agent(query))




Test the General agent and confirm that it consults the KB to answer the questions


In [ ]:
print(answer_general_mortgage_questions("What is the benefit of refinancing, if any?"))

## 5. Create Agent for Existing Mortgage Questions

Create the Agent for managing existing mortgages, for example you can ask when is your next payment due, etc.



In [ ]:
@tool
def get_mortgage_details(customer_id):
    # TODO: Implement real business logic to retrieve mortgage status
    return {
        "account_number": customer_id,
        "outstanding_principal": 150000.0,
        "interest_rate": 4.5,
        "maturity_date": "2030-06-30",
        "payments_remaining": 72,
        "last_payment_date": "2024-06-01",
        "next_payment_due": "2024-07-01",
        "next_payment_amount": 1250.0
    }


In [ ]:
@tool
def answer_existing_mortgage_questions(query):
    # Create the Existing Mortgage Agent
    existing_mortgage_agent = Agent(
        model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
        tools=[
           get_mortgage_details
        ],
        system_prompt="""
        You are an Existing Mortgage Assistant that helps customers with their current mortgages.

        You can:
        1. Provide information about a customer's existing mortgage
        2. Check mortgage status including balance and payment information
        3. Evaluate refinancing eligibility
        4. Calculate payoff timelines with extra payments
        5. Answer questions about mortgage terms and conditions

        When helping users:
        - Always verify the customer ID before providing information
        - Provide clear explanations of mortgage details
        - Format financial data in a readable way
        - Explain payment schedules and upcoming due dates
        - Offer guidance on refinancing options when appropriate
        - Use the knowledge base for detailed information when needed

        Remember that you're dealing with sensitive financial information, so maintain a professional tone
        and ensure accuracy in all responses.
        """
    )
    return str(existing_mortgage_agent(query))



Test the agent to check if it can answer questions about the existing mortgage.

In [ ]:
print(answer_existing_mortgage_questions("I'm customer 98991. when's my next payment due?"))

## 6. Integrate Credit Check Tool via MCP Server

Now let us integrate the credit check tool provided via the MCP server. Strands includes built-in support for connecting to MCP servers and using their tools.



> 🚨 **Important Note:** Before proceeding further **Run the MCP server** provided here [creditcheck_server_http.py](../3_mcp-server-client/strands_mcp/creditcheck_server_http.py). Go to the SageMaker terminal from this notebook and navigate to the **3_mcp-server-client/strands_mcp/** folder and run the below command:

python creditcheck_server_http.py

This will start the MCP server that can accessible through this url: "http://0.0.0.0:8080/mcp".

We will connect to this server through Strands MCP client, list the tools provided by the MCP server and add the tools to the tool list provided the agent as shown int he cell below

In [ ]:
# Create MCP HTTP client
mcp_client = MCPClient(lambda: streamablehttp_client(
    url="http://0.0.0.0:8080/mcp"  # Your MCP server URL
    #streaming=True  # Enable streaming
))


## 7. Create Supervisor Agent

Create the supervisor agent and provide all specialized agent tools as well as the MCP tools. When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools.

In [ ]:
def answer_customer_query(query):
    """
    Process the customer query through the multi-agent system with MCP integration
    
    Args:
        query: The customer's query
        
    Returns:
        str: The response from the supervisor agent
    """
    # Use MCP client context manager to ensure session is active
    with mcp_client:
        try:
            # Get MCP tools
            mcp_tools = mcp_client.list_tools_sync()
            
            # Define supervisor system prompt
            supervisor_system_prompt = """
            Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
            customer questions related to general mortgages questions and queries about the existing mortgage.
            
            For general questions, use the answer_general_mortgage_questions tool.
            For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
            If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
            
            You also have access to MCP tools that can perform additional function to get the credit score of existing customer.
            Use these tools when appropriate for the customer's query.
            
            IMPORTANT: When using credit check tools, return ONLY the credit score value without additional analysis or explanations.
            For other queries, synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
            """
            
            # Combine custom tools with MCP tools
            all_tools = [
                answer_general_mortgage_questions, 
                answer_existing_mortgage_questions, 
                calculator
            ] + mcp_tools
            
            # Create the supervisor agent within MCP context
            supervisor = Agent(
                model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt=supervisor_system_prompt,
                tools=all_tools
            )
            
            # Process the query within the same MCP context
            return supervisor(f"Provide a comprehensive answer for this query: {query}")
            
        except Exception as e:
            logger.error(f"Error processing query with MCP tools: {str(e)}")
            
            # Fallback to supervisor without MCP tools
            supervisor = Agent(
                model="us.anthropic.claude-3-5-haiku-20241022-v1:0",
                system_prompt="""
                Your role is to provide a unified experience for all things related to mortgages. You are a supervisor who oversees answering
                customer questions related to general mortgages questions and queries about the existing mortgage.

                For general questions, use the answer_general_mortgage_questions tool.
                For questions on existing mortgage, use the answer_existing_mortgage_questions tool.
                If asked for a complicated calculation, use your code interpreter to be sure it's done accurately.
                
                Synthesize the details from the response of the tools used into a comprehensive answer provided back to the customer.
                """,
                tools=[answer_general_mortgage_questions, answer_existing_mortgage_questions, calculator]
            )
            
            return supervisor(f"Provide a comprehensive answer for this query: {query}")

## 8. Test the Complete Multi-Agent System

Now let's test the complete multi-agent system with various queries:

In [ ]:
#test the supervisor agent
print("\n\nInvoking supervisor agent...\n\n")

requests = [#"I am customer: 3345, when's my next payment due?",
            "what is my credit score, my customer id is 1111"
            #"what's my balance after that payment, and what rate am I paying?",
            #"why do so many people choose a 30-year mortgage??",
            #"did you receive my employment verification doc yet? i sent it last week",
            #"i'm getting ready to lock in on a rate. what have the rates looked like in last couple weeks?",
            # "great. if i use the highest of those rates for $500K for 15 years, what's my payment?"
            ]

for request in requests:
    print(f"\n\nRequest: {request}\n\n")
    result = answer_customer_query(request)
    time.sleep(10)
    print(result)

## 9. Interactive Chat Loop

Let's create an interactive chat loop to test the system with custom queries:

In [ ]:
import io
import sys

print("Mortgage Assistant Chat - Type 'quit' to exit")
print("=" * 50)

while True:
    user_input = input("\nYou: ").strip()
    
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break
    
    if not user_input:
        continue
    
    try:
        # Capture stdout to prevent double printing
        old_stdout = sys.stdout
        sys.stdout = io.StringIO()
        
        response = answer_customer_query(user_input)
        
        # Restore stdout
        captured_output = sys.stdout.getvalue()
        sys.stdout = old_stdout
        
        print(f"\nAssistant: {response}")
    except Exception as e:
        sys.stdout = old_stdout
        print(f"\nError: {str(e)}")
        print("Please try again or type 'quit' to exit.")


### 🔨 Assignment
 The multi-agent application you created above, just has 2 agents: one to answer general questions on mortgage and the other to answer questions on exiting mortage. You need to add another agent to provide additional functionality to help users initiate a new home loan application by collecting relevant financial information and personal details, just you did leveraging Amazon Bedrock Agent framework in [create-new-application-agent.ipynb](../1_bedrock-single-agent/create-new-application-agent.ipynb)